In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_squared_error as MSE

# データの読み込み
train = pd.read_csv('/content/drive/MyDrive/signate/練習 お弁当の需要予測/train.csv')
test = pd.read_csv('/content/drive/MyDrive/signate/練習 お弁当の需要予測/test.csv')
samplesubmission = pd.read_csv('/content/drive/MyDrive/signate/練習 お弁当の需要予測/sample.csv',header=None)

In [5]:
!apt-get -y install fonts-noto-cjk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  fonts-noto-cjk-extra
The following NEW packages will be installed:
  fonts-noto-cjk
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 61.2 MB of archives.
After this operation, 93.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-noto-cjk all 1:20220127+repack1-1 [61.2 MB]
Fetched 61.2 MB in 3s (21.6 MB/s)
Selecting previously unselected package fonts-noto-cjk.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../fonts-noto-cjk_1%3a20220127+repack1-1_all.deb ...
Unpacking fonts-noto-cjk (1:20220127+repack1-1) ...
Setting up fonts-noto-cjk (1:20220127+repack1-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [10]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager
import matplotlib

In [16]:
y=train['y']
train.drop(['y'], axis=1, inplace=True)
all_data = pd.concat((train, test)).reset_index(drop=True)

In [17]:
all_data["year"]=all_data["datetime"].apply(lambda x:x.split("-")[0]).astype(int)
all_data["month"]=all_data["datetime"].apply(lambda x:x.split("-")[1]).astype(int)
month_dict = {
    '月': 1, '火': 2, '水': 3, '木': 4,
    '金': 5, '土': 6, '日': 7
}
all_data['week'] = all_data['week'].map(lambda x: month_dict.get(str(x).lower())).astype(int)
all_data['precipitation']=all_data['precipitation'].apply(lambda x:0 if x=='--' else float(x))
all_data["remarks"] = all_data.apply(lambda x: 1 if x['remarks'] == 'お楽しみメニュー' and 'カレー' in x['name'] else 0, axis=1)
all_data["weather"] = all_data["weather"].apply(lambda x: "雪" if x == '雷電' else x)
from sklearn.preprocessing import LabelEncoder
for c in ("event","weather"):
    lbl = LabelEncoder()
    lbl.fit(list(all_data[c].values))
    all_data[c] = lbl.transform(list(all_data[c].values))
all_data.drop(["datetime","name"],axis=1,inplace=True)
all_data['temperature'] = all_data['temperature'].apply(lambda x: round(x, 0))

In [19]:
all_data["payday"] = all_data["payday"].fillna(0.5)
all_data["kcal"] = all_data["kcal"].fillna(all_data["kcal"].median())

In [20]:
train =  all_data.iloc[100:207]
test = all_data.iloc[207:, :]

In [21]:
#目的変数及び説明変数を表す変数の準備
y_train = y.iloc[100:207] # 目的変数
X_train = train
X_test =test

model=LR()
model.fit(X_train,y_train)
y_test=model.predict(X_test)
y_test=np.round(y_test).astype(int)

In [22]:
y_test_series = pd.Series(y_test)
result_df = pd.concat([samplesubmission.iloc[:, 0],y_test_series], axis=1)
result_df.to_csv('/content/drive/MyDrive/signate/練習 お弁当の需要予測/submit.csv', header=False, index=False)

In [25]:
train_sub1 = train.head(100)
train_sub2 = train.iloc[100:, :]
sub_y1 = y.head(100)
sub_y2 = y.iloc[100:len(train)]

train_sub1.drop(["event"], axis=1)
train_sub2.drop(["event"], axis=1)

model = LR()
model.fit(train_sub1, sub_y1)
predict1 = model.predict(train_sub2)

# RMSE の計算
RMSE = np.sqrt(MSE(sub_y2, predict1))
print(RMSE)

21.844543558155824
